In [159]:
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from matplotlib import pyplot
import re
from sklearn import preprocessing
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import langid
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
posts = pd.read_csv('InstagramDatasetFinal.csv')

In [2]:
caption = pd.read_csv('CaptionDF.csv')

In [125]:
posts.head()

,biography,business_category_name,connected_fb_page,country_block,edge_felix_video_timeline,edge_follow,edge_followed_by,edge_media_collections,edge_owner_to_timeline_media,edge_saved_media,...,Home Services,Lifestyle Services,Local Events,Non-Profits & Religious Organizations,Personal Goods & General Merchandise Stores,Professional Services,Publishers,Transportation & Accomodation Services,mean_likes,mean_comments
0,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
1,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
2,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
3,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
4,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727


In [139]:
caption1 = posts[['edge_media_to_caption']].copy()

In [140]:
caption1.head(20)

,edge_media_to_caption
0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️
1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️
2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆
3,None
4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?
5,빼꼼! 메이 사랑해요~~~
6,복 많이받으세요!!!!❤️😆
7,이젠 어렵지 않아요
8,다음번에는 예고없이 바로 방송킬꺼에요🤨
9,곧 만나요🐰 먕먕~~!


In [152]:
hashtag_removed = re.sub(r'#[A-Za-z0-9]+','',caption1['edge_media_to_caption'][12])

In [153]:
hashtag_removed

'Hack the game. Unlock New Levels.  '

In [ ]:
from nltk.corpus import stopwords
sw = stopwords.words('english')
np.array(sw)
def stopwords(row):
    words = []
    text = [word.lower()for word in str(row['english']).split() if word.lower() not in sw]
    words.extend(text)
    words = sorted(list(set(words)))
    return words
caption['token'] = caption.apply(stopwords, axis=1)

In [181]:
#block to clean caption text

sw = stopwords.words('english')
np.array(sw)
caption1['caption'] = ''
translator = str.maketrans('', '', string.punctuation)
stemmer= PorterStemmer()
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002700-\U000027BF"
        u"\U00002A00-\U00002AFF"
        u"\U00002600-\U000026FF"
                           "]+", flags=re.UNICODE)
for i in range( 0, caption1['edge_media_to_caption'].size ):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/caption1['edge_media_to_caption'].size*100))+"% done")
    stem_result = []    
    input_str = str(caption1['edge_media_to_caption'][i]).lower() #convert to lowercase
    input_str = emoji_pattern.sub(r'', input_str) #remove emojis
    input_str = re.sub(r'#[A-Za-z0-9]+','',str(input_str)) #remove hashtags
    input_str = re.sub(r'@[A-Za-z0-9]+','',str(input_str)) #remove mentions
    
    #input_str = input_str.translate(translator) # remove punctuation #come back here
    #tokens = word_tokenize(input_str)
    #result = [i for i in tokens if not i in sw] # remove stop words
    #for word in result:
        #stem_result.append(stemmer.stem(word)) #get stem words
    #str1 = ' '.join(stem_result)
    caption1['caption'][i] = input_str
    

5% done
10% done
14% done
19% done
24% done
29% done
34% done
38% done
43% done
48% done
53% done
58% done
63% done
67% done
72% done
77% done
82% done
87% done
91% done
96% done


In [182]:
caption1.head(20)

,edge_media_to_caption,caption
0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️,o'clock이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요️
1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️,즐거운 화이트데이~~메이트데이~~~🧛‍️️
2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆,o'clock 곧 12시죠~! 두구두구 티저 이미지 공개!
3,None,none
4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?
5,빼꼼! 메이 사랑해요~~~,빼꼼! 메이 사랑해요~~~
6,복 많이받으세요!!!!❤️😆,복 많이받으세요!!!!️
7,이젠 어렵지 않아요,이젠 어렵지 않아요
8,다음번에는 예고없이 바로 방송킬꺼에요🤨,다음번에는 예고없이 바로 방송킬꺼에요🤨
9,곧 만나요🐰 먕먕~~!,곧 만나요 먕먕~~!


In [12]:
emoji_regexp = u'[\U00002600-\U000026FF]|[\U00002700-\U000027BF]|[\U0001f300-\U0001f5fF]|[\U0001f600-\U0001f64F]|' \
               u'[\U0001f680-\U0001f6FF]|[\U00002A00-\U00002AFF]|[\U0001F1E0-\U0001F1FF]'
re.findall(emoji_regexp, caption['edge_media_to_caption'][2], re.UNICODE)

['😆']

In [172]:
#caption1['caption'] = re.sub(r'#[A-Za-z0-9]+','',str(caption1['edge_media_to_caption']))
#caption1["caption"] = caption1['edge_media_to_caption'].str.replace('[^\w\s]','')
#caption1["caption"] = caption1['caption'].fillna(" ", inplace = True)
#caption1["caption"] = caption1['caption'].astype(str)

In [183]:
caption1["caption"] = caption1['caption'].str.replace('[^\w\s]','') #remove emojis
caption1.head(20)

,edge_media_to_caption,caption
0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️,oclock이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요
1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️,즐거운 화이트데이메이트데이
2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆,oclock 곧 12시죠 두구두구 티저 이미지 공개
3,None,none
4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요
5,빼꼼! 메이 사랑해요~~~,빼꼼 메이 사랑해요
6,복 많이받으세요!!!!❤️😆,복 많이받으세요
7,이젠 어렵지 않아요,이젠 어렵지 않아요
8,다음번에는 예고없이 바로 방송킬꺼에요🤨,다음번에는 예고없이 바로 방송킬꺼에요
9,곧 만나요🐰 먕먕~~!,곧 만나요 먕먕


In [6]:
def identifyLanguage(row):
    b = TextBlob(str(row['caption']))
    if(len(b) < 3):
        return 'en'
    else:
        return b.detect_language()
#depends on if TextBlob wants to work

In [185]:
def identify_language(row):
    lang = langid.classify(row['caption'])
    return lang[0]
#depends on if langid wants to work

In [186]:
caption1['language'] = caption1.apply(identify_language, axis=1)

In [187]:
caption1.head()

,edge_media_to_caption,caption,language
0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️,oclock이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요,ko
1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️,즐거운 화이트데이메이트데이,ko
2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆,oclock 곧 12시죠 두구두구 티저 이미지 공개,ko
3,None,none,en
4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요,ko


In [188]:
from googletrans import Translator
import pandas as pd
#from py_translator import Translator
translator = Translator()
#df = pd.DataFrame({'Spanish':['piso','cama']})
#df['English'] = df['Spanish'].apply(translator.translate, src='es', dest='en').apply(getattr, args=('text',))

In [13]:
word = translator.translate(caption['caption'][0], src='ko', dest='en').text

In [189]:
def translate(row):
    translator = Translator()
    if str(row['language']) == 'en':
        return str(row['caption'])
    else:
        #return ''
        try:
            english = translator.translate(str(row['caption']), src=row['language'], dest='en').text
            return english
        except:
            return ''

In [190]:
caption1['english'] = ""
#translation to English
for i in range( 0, caption1['caption'].size):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/caption1['caption'].size*100))+"% done")
    if(str(caption1['language'][i]) == 'en'):
        caption1['english'][i] = str(caption1['caption'][i])
    else:
        try:
            english = translator.translate(str(caption1['caption'][i]), src=caption1['language'][i], dest='en').text
            caption1['english'][i] = english
        except:
            caption1['english'][i] = ''

5% done
10% done
14% done
19% done
24% done
29% done
34% done
38% done
43% done
48% done
53% done
58% done
63% done
67% done
72% done
77% done
82% done
87% done
91% done
96% done


In [ ]:
caption['english'] = caption.apply(translate, axis=1)

In [199]:
caption1 = caption1.fillna('')

In [200]:
caption1.head(20)

,edge_media_to_caption,caption,language,english
0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️,oclock이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요,ko,oclock has been released.
1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️,즐거운 화이트데이메이트데이,ko,Happy White Day Mate Day
2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆,oclock 곧 12시죠 두구두구 티저 이미지 공개,ko,oclock shortly 12 o'clock
3,None,none,en,none
4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요,ko,Today is Valentine's Day.
5,빼꼼! 메이 사랑해요~~~,빼꼼 메이 사랑해요,ko,I love you.
6,복 많이받으세요!!!!❤️😆,복 많이받으세요,ko,Happy New Year
7,이젠 어렵지 않아요,이젠 어렵지 않아요,ko,It's not difficult now.
8,다음번에는 예고없이 바로 방송킬꺼에요🤨,다음번에는 예고없이 바로 방송킬꺼에요,ko,Next time we'll air it without notice.
9,곧 만나요🐰 먕먕~~!,곧 만나요 먕먕,ko,See you soon


In [223]:
caption2 = caption1.copy()

In [219]:
caption2 = caption2.drop(columns=['caption','english'])

In [220]:
caption2.head(20)

,edge_media_to_caption,language
0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️,ko
1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️,ko
2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆,ko
3,None,en
4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?,ko
5,빼꼼! 메이 사랑해요~~~,ko
6,복 많이받으세요!!!!❤️😆,ko
7,이젠 어렵지 않아요,ko
8,다음번에는 예고없이 바로 방송킬꺼에요🤨,ko
9,곧 만나요🐰 먕먕~~!,ko


In [221]:
caption2.to_csv('CleanCaptionDF.csv')

In [203]:
caption1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20778 entries, 0 to 20777
Data columns (total 4 columns):
edge_media_to_caption    20778 non-null object
caption                  20778 non-null object
language                 20778 non-null object
english                  20778 non-null object
dtypes: object(4)
memory usage: 649.4+ KB


In [19]:
caption.to_csv('CaptionDF.csv')

# Start of NLP Processing

In [204]:
caption = pd.read_csv('CaptionDF.csv')

In [205]:
caption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20811 entries, 0 to 20810
Data columns (total 5 columns):
Unnamed: 0               20811 non-null object
edge_media_to_caption    20734 non-null object
caption                  20162 non-null object
language                 20771 non-null object
english                  14141 non-null object
dtypes: object(5)
memory usage: 813.0+ KB


In [197]:
import csv
with open(r"CleanCaptionDF.csv", 'rb') as f:
    reader = csv.reader(f)
    linenumber = 1
    try:
        for row in reader:
            linenumber += 1
    except Exception as e:
        print ("Error line %d: %s %s" % (linenumber, str(type(e)), e.message))

AttributeError: 'Error' object has no attribute 'message'

In [222]:
caption = pd.read_csv('CleanCaptionDF.csv')

In [82]:
caption.head()

,Unnamed: 0,edge_media_to_caption,caption,language,english
0,0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️,OCLOCK이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요,ko,OCLOCK has been released.
1,1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️,즐거운 화이트데이메이트데이,ko,Happy White Day Mate Day
2,2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆,OCLOCK 곧 12시죠 두구두구 티저 이미지 공개,ko,OCLOCK soon to be 12 o'clock
3,3,None,None,en,None
4,4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요,ko,Today is Valentine's Day.


In [ ]:
sw = stopwords.words('english')
np.array(sw)
caption1['caption'] = ''
translator = str.maketrans('', '', string.punctuation)
stemmer= PorterStemmer()

for i in range( 0, caption1['edge_media_to_caption'].size ):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/caption1['edge_media_to_caption'].size*100))+"% done")
    stem_result = []    
    input_str = str(caption1['edge_media_to_caption'][i]).lower() #convert to lowercase
    #input_str = emoji_pattern.sub(r'', input_str) #remove emojis
    #input_str = re.sub(r'#[A-Za-z0-9]+','',str(input_str)) #remove hashtags
    #input_str = re.sub(r'@[A-Za-z0-9]+','',str(input_str)) #remove mentions
    
    input_str = input_str.translate(translator) # remove punctuation #come back here
    tokens = word_tokenize(input_str)
    result = [i for i in tokens if not i in sw] # remove stop words
    for word in result:
        stem_result.append(stemmer.stem(word)) #get stem words
    str1 = ' '.join(stem_result)
    caption1['caption'][i] = input_str

In [83]:
from nltk.corpus import stopwords
sw = stopwords.words('english')
np.array(sw)
def stopwords(row):
    words = []
    text = [word.lower()for word in str(row['english']).split() if word.lower() not in sw]
    words.extend(text)
    words = sorted(list(set(words)))
    return words
caption['token'] = caption.apply(stopwords, axis=1)

In [84]:
caption['english'] = caption['english'].replace(np.nan, '', regex=True)

In [85]:
caption.head(20)

,Unnamed: 0,edge_media_to_caption,caption,language,english,token
0,0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️,OCLOCK이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요,ko,OCLOCK has been released.,"[oclock, released.]"
1,1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️,즐거운 화이트데이메이트데이,ko,Happy White Day Mate Day,"[day, happy, mate, white]"
2,2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆,OCLOCK 곧 12시죠 두구두구 티저 이미지 공개,ko,OCLOCK soon to be 12 o'clock,"[12, o'clock, oclock, soon]"
3,3,None,None,en,None,[none]
4,4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요,ko,Today is Valentine's Day.,"[day., today, valentine's]"
5,5,빼꼼! 메이 사랑해요~~~,빼꼼 메이 사랑해요,ko,I love you.,"[love, you.]"
6,6,복 많이받으세요!!!!❤️😆,복 많이받으세요,ko,Happy New Year,"[happy, new, year]"
7,7,이젠 어렵지 않아요,이젠 어렵지 않아요,ko,It's not difficult now.,"[difficult, now.]"
8,8,다음번에는 예고없이 바로 방송킬꺼에요🤨,다음번에는 예고없이 바로 방송킬꺼에요,ko,Next time we'll air it without notice.,"[air, next, notice., time, we'll, without]"
9,9,곧 만나요🐰 먕먕~~!,곧 만나요 먕먕,ko,See you soon,"[see, soon]"


In [86]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [87]:
count_vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 100) 

In [88]:
sentences = []
for i in range( 0, caption['english'].size ):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/30000*100))+"% done")
    sentences.append(caption['english'][i])

3% done
7% done
10% done
13% done
17% done
20% done
23% done
27% done
30% done
33% done
37% done
40% done
43% done
47% done
50% done
53% done
57% done
60% done
63% done
67% done


In [28]:
word_features = count_vectorizer.fit_transform(sentences)

#tfidf_transformer = TfidfTransformer()
#clean_features = tfidf_transformer.fit_transform(clean_features)

# Numpy arrays are easy to work with, so convert the result to an 
# array
word_features = word_features.toarray()

In [46]:
vocab = count_vectorizer.get_feature_names()
word_features = pd.DataFrame(data=word_features, columns=vocab)
len(word_features)

20811

In [34]:
from textblob import TextBlob

In [225]:
caption2['sentiment'] = ''
for i in range( 0, caption2['english'].size):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/caption2['english'].size*100))+"% done")
    analysis = TextBlob(caption2['english'][i])
    caption2['sentiment'][i] = analysis.sentiment[0]

5% done
10% done
14% done
19% done
24% done
29% done
34% done
38% done
43% done
48% done
53% done
58% done
63% done
67% done
72% done
77% done
82% done
87% done
91% done
96% done


In [228]:
caption2.head()

,edge_media_to_caption,caption,english,sentiment,language_af,language_an,language_ar,language_az,language_be,language_bg,...,language_tr,language_ug,language_uk,language_ur,language_vi,language_vo,language_wa,language_xh,language_zh,language_zu
0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️,oclock이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요,oclock has been released.,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️,즐거운 화이트데이메이트데이,Happy White Day Mate Day,0.4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆,oclock 곧 12시죠 두구두구 티저 이미지 공개,oclock shortly 12 o'clock,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,None,none,none,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요,Today is Valentine's Day.,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
caption['language'] = caption['language'].replace(np.nan, 'en', regex=True)
caption['edge_media_to_caption'] = caption['edge_media_to_caption'].replace(np.nan, 'en', regex=True)
caption['caption'] = caption['caption'].replace(np.nan, 'en', regex=True)

In [112]:
caption = pd.get_dummies(caption,columns=['language'])

In [227]:
caption2 = pd.get_dummies(caption2,columns=['language'])

In [230]:
nlp_features = caption2.copy()
#nlp_features = caption.drop(caption.columns[cols],axis=1,inplace=True)

In [231]:
cols = [0,1,2]
nlp_features.drop(nlp_features.columns[cols],axis=1,inplace=True)

In [232]:
nlp_features.head()

,sentiment,language_af,language_an,language_ar,language_az,language_be,language_bg,language_br,language_bs,language_ca,...,language_tr,language_ug,language_uk,language_ur,language_vi,language_vo,language_wa,language_xh,language_zh,language_zu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [233]:
nlp_features.to_csv('NLPFeatures.csv')